In [24]:
import random
import string

import numpy as np
import scipy as sp

import pandas as pd

from sklearn.model_selection import StratifiedShuffleSplit

import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
def generate_string(lenght):
    return ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase, k=lenght))

def categorical_noise(column, ammount):
    ammount = int(len(column) * ammount)
    sample = random.choices(column.index, k=ammount)
    string = generate_string(5)
    column.loc[sample] = column.loc[sample].apply(lambda x: string)
    return column

def scalar_noise(column, ammount):
    ammount = int(len(column) * ammount)
    sample = random.choices(column.index, k=ammount)
    std = column.std()
    column.loc[sample] = column.loc[sample].apply(lambda x: x + std*10)
    return column
        
def add_noise (df, n_columns):
    columns = list(df)
    for column in random.choices(columns, k=n_columns):
        
        if df[column].dtype == 'object':
            df[column] = categorical_noise(df[column], 0.5)
        elif df[column].dtype in ['int64', 'int32', 'float64', 'float32']:
            df[column] = scalar_noise(df[column], 0.5)
    
    return df


In [26]:
df = pd.read_csv('C:\\Users\\penthotal\\Desktop\\IGGI\\collaborations\\personal_projects\\data_viz_python\\data\\dataframe.csv')
noise_df = df = add_noise(df, 5)

c:\users\penthotal\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


WHAT, WHY, WHERE, HOW DATA VISUALIZATION

BRIEF INTRODUCTION PANDAS DATAFRAMES, MATPLOTLIB AND SEABORN

EXPLORATORY DATA VISUALIZATION

* distplot and count plot
* ridge plot
* Joint kernel density estimate

EXERCISE 1

* discover the characteristics of the dataframe you are working on
* discover anomalies in the data
* revelation of the data source

DATA VISUALIZATION FOR HYPOTHESIS TESTING
* line plot
* scatter plot and confusion matrix
* boxplot

EXERCISE 2
* find and test your own hp!

DATA VISUALIZATION FOR MODEL EVALUATION
* model performance (line plot, box plot)
* model accuracy (line plot, boxplot, confusion matrix)
* model inspection (scatter plot)

EXERCISE 3